In [1]:
from huggingface_hub import notebook_login
notebook_login()

In [2]:
import transformers

In [3]:
import json
with open("../src/outputs/blobs_tokenized_timit_converted.json", "r") as f:
    tokenized_dataset = json.load(f)

# Create a DatasetDict from the tokenized json dataset
from datasets import Dataset
from datasets.dataset_dict import DatasetDict
audio_tokens = []
attention_mask = []
for i, utterance in enumerate(tokenized_dataset):
    if (len(utterance) < 1024):
        continue
    else:
      audio_tokens.append(utterance[:1024])
      attention_mask.append([1] * len(utterance[:1024]))


tokenized_datasets = DatasetDict({
    "train": Dataset.from_dict({"input_ids": audio_tokens[:300], "attention_mask": attention_mask[:300]}),
    "validation": Dataset.from_dict({"input_ids": audio_tokens[300:350], "attention_mask": attention_mask[300:350]}),
    "test": Dataset.from_dict({"input_ids": audio_tokens[350:380], "attention_mask": attention_mask[350:380]})
})
tokenized_datasets
tokenized_datasets["train"][1]

{'input_ids': [1936,
  63458,
  5000,
  63457,
  63426,
  2635,
  669,
  341,
  9964,
  384,
  272,
  2939,
  29603,
  1014,
  490,
  63443,
  18,
  31841,
  63458,
  63423,
  19766,
  63889,
  63762,
  63649,
  63473,
  63987,
  63978,
  63924,
  19766,
  53217,
  63762,
  63649,
  63598,
  63857,
  63978,
  63167,
  19766,
  53217,
  63762,
  63649,
  63598,
  63962,
  63978,
  63924,
  19766,
  53217,
  63762,
  63717,
  63598,
  63962,
  63978,
  63924,
  51535,
  63513,
  63654,
  63717,
  56387,
  63857,
  63546,
  63167,
  51535,
  63513,
  63654,
  63717,
  56387,
  63915,
  63546,
  63951,
  51535,
  63889,
  63913,
  63649,
  56387,
  55128,
  63546,
  63951,
  51535,
  63889,
  63913,
  63649,
  56387,
  63915,
  63546,
  63924,
  19766,
  53217,
  63762,
  63649,
  63598,
  63962,
  63978,
  63951,
  51535,
  63889,
  63913,
  63717,
  56387,
  63915,
  63546,
  63924,
  63795,
  63970,
  63629,
  63830,
  63920,
  63861,
  63775,
  25046,
  63811,
  53217,
  63578,
  43435

In [4]:
from transformers import AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained("distilgpt2")

In [5]:
from transformers import Trainer, TrainingArguments

In [6]:
model_name = "distilgpt2" 
training_args = TrainingArguments(
    f"{model_name}-timit",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=True,
)

In [7]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
)

c:\Users\Admin\Documents\GitHub\transformer-audio\gpt-sw3\distilgpt2-timit is already a clone of https://huggingface.co/anforsm/distilgpt2-timit. Make sure you pull the latest changes with `repo.git_pull()`.


In [8]:
trainer.train()

c:\Python38\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 300
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 114
  Number of trainable parameters = 81912576
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: anforsm (dt2112g1). Use `wandb login --relogin` to force relogin


  0%|          | 0/114 [00:00<?, ?it/s]

RuntimeError: CUDA error: CUBLAS_STATUS_NOT_INITIALIZED when calling `cublasCreate(handle)`

: 